In [25]:
import scipy.special as sp
import numba
import numba.cuda
import numpy as np
import accelerate
from __future__ import division

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 24 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 24 days


In [37]:
def testCalc(A, x, C):
    for i in range(A.shape[0]):
        C[i] = sp.gammainc(A[i], x[i])
        
@cuda.jit
def testCalcJIT(A, x, C):
    pos = cuda.grid(1)
    if pos < A.shape[0]:
        C[pos] = A[pos] + x[pos]
        
@numba.vectorize(['float32(float32, float32)',
                     'float64(float64, float64)'], target='cuda')
def gpu_sincos(x, y):
    return math.sin(x) * math.cos(y)
        
@numba.guvectorize(['void(float32[:], float32[:], float32[:])'],'(n),(n)->(n)', target='cuda')
def batch_matrix_mult(a, x, c):
    for i in range(c.shape[0]):
        c[i] = np.random.normal(a[i], x[i])

UntypedAttributeError: Caused By:
Traceback (most recent call last):
  File "/home/bl/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 243, in run
    res = stage()
  File "/home/bl/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 463, in stage_nopython_frontend
    self.locals)
  File "/home/bl/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 780, in type_inference_stage
    infer.propagate()
  File "/home/bl/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 565, in propagate
    raise errors[0]
UntypedAttributeError: Unknown attribute "normal" of type Module(<module 'numpy.random' from '/home/bl/anaconda2/lib/python2.7/site-packages/numpy/random/__init__.pyc'>)
File "<ipython-input-37-cd532b7ccf86>", line 19

Failed at nopython (nopython frontend)
Unknown attribute "normal" of type Module(<module 'numpy.random' from '/home/bl/anaconda2/lib/python2.7/site-packages/numpy/random/__init__.pyc'>)
File "<ipython-input-37-cd532b7ccf86>

In [21]:
n = 100
A = np.random.rand(n)
x = np.random.rand(n)
C = np.empty(n)

an_array = np.ones((n,n))

threadsperblock = (16, 16)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
increment_a_2D_array[blockspergrid, threadsperblock](an_array)

ArgumentError: argument 2: <type 'exceptions.TypeError'>: wrong type

In [12]:
%%timeit
testCalc(A, x, C)

100 loops, best of 3: 11.9 ms per loop


In [14]:
%%timeit
testCalcJIT(A, x, C)

100 loops, best of 3: 11.5 ms per loop
